In [ ]:
import pandas as pd
import numpy as np
from scipy.cluster.hierarchy import dendrogram, linkage      
import sklearn.feature_extraction.text as sk_text
import sklearn.cluster as sk_cluster
import sklearn.metrics as metrics
%matplotlib inline
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt
pd.set_option('display.max_colwidth', -1)
import datetime
startTime = datetime.datetime.now()
print(str(startTime))

In [ ]:
#print initial data

tweets = pd.read_csv('clean_data.csv',dtype=str)
tweets['FrequencyOver20'] = tweets.FrequencyOver20.str.lower()
tweets.head()


In [ ]:
aggregateTweetsHashtags = tweets.groupby('UserID')['FrequencyOver20'].apply(lambda x: x.str.cat(sep=' '))
tweetsPrepareSKText = pd.DataFrame({'User_id': aggregateTweetsHashtags.index, 'All_hashtags': aggregateTweetsHashtags.values})
vectorizer = sk_text.TfidfVectorizer(max_features = 4,
                             #min_df=100, 
                             max_df=.8,
                             stop_words = 'english'
                             )


In [ ]:
matrix = vectorizer.fit_transform(tweetsPrepareSKText.All_hashtags.values)
tdidf = matrix.toarray()
df_text = pd.DataFrame(matrix.todense(), index=aggregateTweetsHashtags.index, columns=vectorizer.get_feature_names())
df_text.head()

In [ ]:
ground_cols = ['UserID', 'GROUND_TRUTH_TEAM']
ground_truth = pd.read_table('clinton_trump_user_classes.txt', encoding ="ISO-8859-1", dtype=str, names= ground_cols)
#ground_truth.UserID = ground_truth.UserID.astype(int)
df_text = pd.merge(df_text, ground_truth, on = 'UserID')
#ground truth
true = df_text['GROUND_TRUTH_TEAM'].astype(int).values

In [ ]:
#Cluster centers is acs_order_centroids
def printClusters(vectorizer,cluster_centers,num_cluster,num_to_print):
    cluster_centers_reversed = cluster_centers[:,::-1]
    des_order_centroids = cluster_centers.argsort()[:,::-1]     #  get the indices that sort array in descending order
    terms = vectorizer.get_feature_names()

    for i in range(num_cluster):
        print ("Cluster:", i)
        for indx,ind in enumerate(des_order_centroids[i, : num_to_print]):
            print (terms[ind],cluster_centers_reversed[i,indx],ind )
        print()


In [ ]:
def printMetrics(true,kmeans):
    print('confusion matrix\n',metrics.confusion_matrix(true, kmeans.labels_, labels=[0, 1]))
    print('Precision',metrics.precision_score(true, kmeans.labels_, average='weighted')) # weighted: the average precision of all clusters is returned
    print('Recall',metrics.recall_score(true, kmeans.labels_, average='weighted'))  # weighted: the average recall of all clusters is returned
    print('F1 Score',metrics.f1_score(true, kmeans.labels_, average= 'weighted'))     # weighted: the average f1 of all clusters is returned


**K-means Clustering**

In [ ]:
%%time
kmeans = sk_cluster.KMeans(n_clusters=2, n_init=20, max_iter=200)  
km_labels = kmeans.fit_predict(tdidf)
error = kmeans.inertia_       #SSE; Sum of squared distances of tsamples to their closest cluster center.

In [ ]:
print ("\nThe total error of the clustering is: ", error)

In [ ]:
printMetrics(true,kmeans)

In [ ]:
printClusters(vectorizer, kmeans.cluster_centers_,2,30)

In [ ]:
print(vectorizer.get_feature_names())

In [ ]:
kmeans.cluster_centers_[:,::-1]


In [ ]:
kmeans.cluster_centers_.argsort()[:,::-1]

**MAX-Agglomerative Clustering**

In [ ]:
## parameter “linkage” to “complete” gives you MAX-based agglomerative hierarchical clustering

ag = sk_cluster.AgglomerativeClustering(linkage = 'complete', n_clusters = 2)   
ag_labels = ag.fit_predict(tdidf)

printMetrics(true,ag)

**SSE-Agglomerative Clustering**

In [ ]:
## parameter “linkage” to “ward” gives you SSE-based agglomerative hierarchical clustering

ag = sk_cluster.AgglomerativeClustering(linkage = 'ward', n_clusters = 2)   
ag_labels = ag.fit_predict(tdidf)

printMetrics(true,ag)

## Task 1.3 (10 pts): Show the two respective word clouds of the two centers (centroids) by using hashtags/handles and their tfidf values. Hint: Use function fit_words() that comes with wordcloud

In [ ]:
# Create stopword list:
stopwords = set(STOPWORDS)

# Generate a word cloud image
wordcloud = WordCloud(stopwords=stopwords).fit_words(vectorizer.vocabulary_)

# Display the generated image:
plt.figure(figsize=(10,5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()

In [ ]:
ag = sk_cluster.AgglomerativeClustering(linkage = 'ward', n_clusters = 3)   

#complete: The maximum distances
#n_clusters: The number of clusters to find.

ag_labels = ag.fit_predict(tdidf)

print ('\nPrinting cluster assignment:')
ag_labels 

## 2.1: First, you apply the k-means algorithm. Create a plot of the SSE error of the k-means algorithm as a function of the number of clusters, for k up to 20, in order to determine the optimal number of clusters.


In [ ]:
error = np.zeros(30)
for k in range(20,30):
    kmeans = sk_cluster.KMeans(n_clusters=k, n_init=10, max_iter=300)
    kmeans.fit_predict(tdidf)
    error[k] = kmeans.inertia_

plt.plot(range(20, 30), error[20:])
plt.xlabel('Number of clusters')
plt.ylabel('SSE')

## 2.2: Run the k-means algorithm for the optimal number of clusters you identified in the last task. Print some hashtags/handles in each cluster. From the hashtags/handles in each cluster, try to deduce what is the topic it concerns. Include your conclusions in your report.

In [ ]:
kmeans = sk_cluster.KMeans(n_clusters=20, n_init=10, max_iter=300)
kmeans.fit_predict(tdidf)
printClusters(vectorizer, kmeans.cluster_centers_, 20,30)

In [ ]:
vectorizer.vocabulary_